In [2]:
# weather_scrape_roll100.py
import requests
import pandas as pd
import time
import os
from datetime import datetime

HEADERS = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 Chrome/117.0 Safari/537.36"}

def fetch_wttr(city):
    url = f"https://wttr.in/{city}?format=j1"
    resp = requests.get(url, headers=HEADERS, timeout=15)
    resp.raise_for_status()
    data = resp.json()

    rows = []
    fetched_at = datetime.utcnow().isoformat()

    # Current condition
    try:
        cc = data["current_condition"][0]
        rows.append({
            "city": city,
            "date": datetime.now().strftime("%Y-%m-%d"),
            "type": "current",
            "temp_C": cc.get("temp_C"),
            "humidity_pct": cc.get("humidity"),
            "wind_kmph": cc.get("windspeedKmph"),
            "min_tempC": None,
            "max_tempC": None,
            "weather_desc": cc.get("weatherDesc", [{}])[0].get("value", ""),
            "fetched_at": fetched_at,
            "source": "wttr.in"
        })
    except Exception:
        pass

    # Daily forecasts
    for day in data.get("weather", []):
        date = day.get("date")
        mint = day.get("mintempC")
        maxt = day.get("maxtempC")
        hourly = day.get("hourly", [])
        if hourly:
            hums = [int(h.get("humidity", 0)) for h in hourly]
            winds = [int(h.get("windspeedKmph", 0)) for h in hourly]
            avg_hum = round(sum(hums) / len(hums), 1)
            avg_wind = round(sum(winds) / len(winds), 1)
            desc = hourly[0].get("weatherDesc", [{}])[0].get("value", "")
        else:
            avg_hum = None
            avg_wind = None
            desc = ""

        rows.append({
            "city": city,
            "date": date,
            "type": "forecast",
            "temp_C": None,
            "humidity_pct": avg_hum,
            "wind_kmph": avg_wind,
            "min_tempC": mint,
            "max_tempC": maxt,
            "weather_desc": desc,
            "fetched_at": fetched_at,
            "source": "wttr.in"
        })

    return rows

def save_to_csv(rows, filename="weather_roll100.csv"):
    df = pd.DataFrame(rows)
    if df.empty:
        print("No data to save.")
        return
    if not os.path.exists(filename):
        df.to_csv(filename, index=False)
    else:
        df.to_csv(filename, mode="a", header=False, index=False)
    print(f"Saved {len(df)} rows to {filename}")

def main():
    
    cities = ["Gurgaon", "Delhi", "Noida","Rajasthan","Hyderabad","Punjab","Gorakhpur","Goa","Mumbai","Jaipur","Jammu","Kerala"]
    all_rows = []
    for city in cities:
        try:
            print("Fetching", city)
            rows = fetch_wttr(city)
            all_rows.extend(rows)
        except Exception as e:
            print("Error for", city, e)
        time.sleep(1)  # polite delay

    save_to_csv(all_rows, filename="weather_roll100.csv")

if __name__ == "__main__":
    main()


Fetching Gurgaon
Fetching Delhi
Fetching Noida
Fetching Rajasthan
Fetching Hyderabad
Fetching Punjab
Fetching Gorakhpur
Fetching Goa
Fetching Mumbai
Fetching Jaipur
Fetching Jammu
Fetching Kerala
Saved 48 rows to weather_roll100.csv
